In [1]:
import pandas as pd
import numpy as np
from loaders import *
import math
import random

In [2]:
matrix_size = 4096

dataflow_permuations = [['M', 'N', 'K'], ['M', 'K', 'N'], ['N', 'K', 'M'], ['N', 'M', 'K'], ['K', 'N', 'M'], ['K', 'M', 'N']]
'''
6 different dataflow permutations
9 components for dataflow
= 6^9 = 10077696 possible combinations
'''
# generate all the data flow permuations
def get_dataflow_permutations():
    dataflow_perumations = []
    
    for DRAM in dataflow_permuations:
        for ClusterArray in dataflow_permuations:
            for GLB_Cluster in dataflow_permuations:
                for glb in dataflow_permuations:
                    for PE_Cluster in dataflow_permuations:
                        for iact_spad in dataflow_permuations:
                            for weight_spad in dataflow_permuations:
                                for psum in dataflow_permuations:
                                    for reg in dataflow_permuations:
                                        permutation = {
                                            "DRAM": DRAM,
                                            "ClusterArray": ClusterArray,
                                            "GLB_Cluster": GLB_Cluster,
                                            "glb": glb,
                                            "PE_Cluster": PE_Cluster,
                                            "iact_spad": iact_spad,
                                            "weight_spad": weight_spad,
                                            "psum": psum,
                                            "reg": reg,
                                        }
                                        dataflow_perumations.append(permutation)
    return dataflow_perumations

# generate all the possible factor permutations for a specific rank (ie, M, N, or K)
def get_factor_permutations():
    factor_permutations = []
    
    for DRAM in [2**i for i in range(int(math.log2(matrix_size)) + 1)]:
        curr_prod1 = DRAM
        for ClusterArray in [2**i for i in range(int(math.log2(matrix_size // curr_prod1)) + 1)]:
            curr_prod2 = curr_prod1 * ClusterArray
            for GLB_Cluster in [2**i for i in range(int(math.log2(matrix_size // curr_prod2)) + 1)]:
                curr_prod3 = curr_prod2 * GLB_Cluster
                for glb in [2**i for i in range(int(math.log2(matrix_size // curr_prod3)) + 1)]:
                    curr_prod4 = curr_prod3 * glb
                    for PE_Cluster in [2**i for i in range(int(math.log2(matrix_size // curr_prod4)) + 1)]:
                        curr_prod5 = curr_prod4 * PE_Cluster
                        for iact_spad in [2**i for i in range(int(math.log2(matrix_size // curr_prod5)) + 1)]:
                            curr_prod6 = curr_prod5 * iact_spad
                            for weight_spad in [2**i for i in range(int(math.log2(matrix_size // curr_prod6)) + 1)]:
                                curr_prod7 = curr_prod6 * weight_spad
                                for psum in [2**i for i in range(int(math.log2(matrix_size // curr_prod7)) + 1)]:
                                    curr_prod8 = curr_prod7 * psum
                                    for reg in [2**i for i in range(int(math.log2(matrix_size // curr_prod8)) + 1)]:
                                        final_prod = curr_prod8 * reg
                                        if final_prod == matrix_size:
                                            permuation = {
                                                "DRAM": DRAM,
                                                "ClusterArray": ClusterArray,
                                                "GLB_Cluster": GLB_Cluster,
                                                "glb": glb,
                                                "PE_Cluster": PE_Cluster,
                                                "iact_spad": iact_spad,
                                                "weight_spad": weight_spad,
                                                "psum": psum,
                                                "reg": reg,
                                            }
                                            factor_permutations.append(permuation)
                                            
    return factor_permutations

In [15]:
dataflow_permutations = get_dataflow_permutations()
factor_permutations = get_factor_permutations()

In [19]:
def evaluate_config(config):
    print(config)
    out = run_timeloop_model(
        config,
        problem='designs/baseline/baseline_problem.yaml',
        mapping='designs/baseline/baseline_mapping.yaml',
        sparse_optimizations='designs/baseline/baseline_sparse_opt.yaml', 
    )
    stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
    print(stats)

def hash_config(config):
    return tuple((k, tuple(v) if isinstance(v, list) else v) for k, v in config.items())

def sample_config(dataflow_permutations, factor_permutations):
    M_factor_permutation = random.choice(factor_permutations)
    N_factor_permutation = random.choice(factor_permutations)
    K_factor_permutation = random.choice(factor_permutations)
    
    dataflow_permutation = random.choice(dataflow_permutations)
    
    config = dict(
        matrix_size_M_dim=matrix_size,
        matrix_size_K_dim=matrix_size,
        matrix_size_N_dim=matrix_size,
        
        DRAM_factor_M=M_factor_permutation["DRAM"],
        DRAM_factor_N=N_factor_permutation["DRAM"],
        DRAM_factor_K=K_factor_permutation["DRAM"],
        DRAM_permutation=dataflow_permutation["DRAM"],
    
        ClusterArray_factor_M=M_factor_permutation["ClusterArray"],
        ClusterArray_factor_N=N_factor_permutation["ClusterArray"],
        ClusterArray_factor_K=K_factor_permutation["ClusterArray"],
        ClusterArray_permutation=dataflow_permutation["ClusterArray"],
    
        GLB_Cluster_factor_M=M_factor_permutation["GLB_Cluster"],
        GLB_Cluster_factor_N=N_factor_permutation["GLB_Cluster"],
        GLB_Cluster_factor_K=K_factor_permutation["GLB_Cluster"],
        GLB_Cluster_permutation=dataflow_permutation["GLB_Cluster"],
    
        glb_factor_M=M_factor_permutation["glb"],
        glb_factor_N=N_factor_permutation["glb"],
        glb_factor_K=K_factor_permutation["glb"],
        glb_permutation=dataflow_permutation["glb"],
    
        PE_Cluster_factor_M=M_factor_permutation["PE_Cluster"],
        PE_Cluster_factor_N=N_factor_permutation["PE_Cluster"],
        PE_Cluster_factor_K=K_factor_permutation["PE_Cluster"],
        PE_Cluster_permutation=dataflow_permutation["PE_Cluster"],
    
        iact_spad_factor_M=M_factor_permutation["iact_spad"],
        iact_spad_factor_N=N_factor_permutation["iact_spad"],
        iact_spad_factor_K=K_factor_permutation["iact_spad"],
        iact_spad_permutation=dataflow_permutation["iact_spad"],
    
        weight_spad_factor_M=M_factor_permutation["weight_spad"],
        weight_spad_factor_N=N_factor_permutation["weight_spad"],
        weight_spad_factor_K=K_factor_permutation["weight_spad"],
        weight_spad_permutation=dataflow_permutation["weight_spad"],
    
        psum_factor_M=M_factor_permutation["psum"],
        psum_factor_N=N_factor_permutation["psum"],
        psum_factor_K=K_factor_permutation["psum"],
        psum_permutation=dataflow_permutation["psum"],
    
        reg_factor_M=M_factor_permutation["reg"],
        reg_factor_N=N_factor_permutation["reg"],
        reg_factor_K=K_factor_permutation["reg"],
        reg_permutation=dataflow_permutation["reg"],
    
        density_weights=1,
        density_inputs=1
    )
    return config

In [20]:
num_of_random_samples = 2
visited = set()

# uses random sampling instead of a for loop
for sample in range(num_of_random_samples):
    # sample
    config = sample_config(dataflow_permutations, factor_permutations)
    while hash_config(config) in visited:
        config = sample_config(dataflow_permutations, factor_permutations)
    evaluate_config(config)

    visited.add(hash_config(config))

{'matrix_size_M_dim': 4096, 'matrix_size_K_dim': 4096, 'matrix_size_N_dim': 4096, 'DRAM_factor_M': 4, 'DRAM_factor_N': 4, 'DRAM_factor_K': 8, 'DRAM_permutation': ['K', 'M', 'N'], 'ClusterArray_factor_M': 8, 'ClusterArray_factor_N': 1, 'ClusterArray_factor_K': 4, 'ClusterArray_permutation': ['K', 'N', 'M'], 'GLB_Cluster_factor_M': 1, 'GLB_Cluster_factor_N': 2, 'GLB_Cluster_factor_K': 1, 'GLB_Cluster_permutation': ['N', 'M', 'K'], 'glb_factor_M': 4, 'glb_factor_N': 8, 'glb_factor_K': 8, 'glb_permutation': ['K', 'N', 'M'], 'PE_Cluster_factor_M': 1, 'PE_Cluster_factor_N': 1, 'PE_Cluster_factor_K': 2, 'PE_Cluster_permutation': ['N', 'M', 'K'], 'iact_spad_factor_M': 1, 'iact_spad_factor_N': 4, 'iact_spad_factor_K': 1, 'iact_spad_permutation': ['K', 'M', 'N'], 'weight_spad_factor_M': 4, 'weight_spad_factor_N': 4, 'weight_spad_factor_K': 8, 'weight_spad_permutation': ['K', 'M', 'N'], 'psum_factor_M': 8, 'psum_factor_N': 4, 'psum_factor_K': 1, 'psum_permutation': ['M', 'K', 'N'], 'reg_factor_M'

INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/final_project/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


Exception: 

========================================================================================================================
Timeloop model failed with return code 1. Please check the output files in ./output_dir for more information. To debug, you can edit the file:
	./output_dir/parsed-processed-input.yaml
and run 
	timeloop model ./output_dir/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.

In [21]:
!timeloop model ./output_dir/parsed-processed-input.yaml

Running apps: model
Found parsed-processed-input.yaml in input files. Running Timeloop without parsing or processing steps. If this is not the intended behavior, please name the input files differently.
input file: /home/workspace/final_project/output_dir/parsed-processed-input.yaml
execute:/usr/local/bin/accelergy /home/workspace/final_project/output_dir/parsed-processed-input.yaml --oprefix timeloop-model. -o ./ > timeloop-model.accelergy.log 2>&1
ERROR: parsing mapping: product of all factors of dimension N is 2048, which is not equal to the dimension size of the workload 4096.
